In [1]:
from time import time
import random

### Evaluation function 

In [2]:
def evaluate_solution(solution, books_scores):
    books_scanned = set()
    points = 0
    number_of_libraries = solution[0]
    #print("Num of libraries: ", number_of_libraries)
    for i in range(1, number_of_libraries+1):
        number_of_books = solution[i][1]
        #print("Num of books in libr: ",i, ":", number_of_books)
        for j in range(number_of_books):
            book_to_add = solution[i][2][j]
            if book_to_add not in books_scanned:
                books_scanned.add(book_to_add)
                #print("add: ", book_to_add)
    #print("Books scanned: ", books_scanned)
    #print("Books scores: ", books_scores)
    solution_score = 0
    for i in books_scanned:
        solution_score += books_scores[i]
        
    return solution_score

### Get complete solution from list of solutions

In [3]:
def get_complete_solution(solution_indices, libraries, days_number, books_scores):
    time_pointer = days_number
    used_books = set()
    complete_solution = [len(solution_indices)]
    for i in solution_indices:
        time_pointer -= libraries[i][0][1]
        books_per_day = libraries[i][0][2]
        available_books = list(libraries[i][1])
        particular_solution = [i, 0, []]
        available_books.sort(key = (lambda x: books_scores[x]))
        for x in available_books:
            if len(particular_solution[2]) > time_pointer * books_per_day:
                break
            if x not in used_books:
                particular_solution[2].append(x)
                particular_solution[1] += 1
                used_books.add(x)
        particular_solution[2] = tuple(particular_solution[2])
        complete_solution.append(tuple(particular_solution))
    return tuple(complete_solution)

### Initial population

In [4]:
def get_random_solution(libraries_number, days_number, libraries, books_scores):
    solution_indices = []
    time_pointer = 0
    while True:
        random_index = random.choice(list(range(libraries_number)))
        random_library = libraries[random_index]
        if time_pointer + random_library[0][1] > days_number:
            break                                 
        if random_index not in solution_indices:
            solution_indices.append(random_index)
            time_pointer += random_library[0][1]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)
        
    
def get_initial_population(population_size, libraries_number, days_number, libraries, books):
    population = []
    for _ in range(population_size):
        population.append(get_random_solution(libraries_number, days_number, libraries, books))
    return population    

### Mutation

In [5]:
def mutate(solution, libraries, days_number, books_scores):
    solution_indices = [solution[i][0] for i in range(1, len(solution))]
    nucleotide_a = random.randint(0, len(solution_indices) - 1)
    nucleotide_b = random.randint(0, len(solution_indices) - 1)
    solution_indices[nucleotide_a], solution_indices[nucleotide_b] = solution_indices[nucleotide_b], solution_indices[nucleotide_a]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)

def do_mutations(population, libraries, days_number, books_scores):
    mutated_solutions = []
    for solution in population:
        mutated_solutions.append(mutate(solution, libraries, days_number, books_scores))
    return mutated_solutions

## Crossover

In [6]:
def get_children(parent_a, parent_b, libraries, days_number, books_scores):
    parent_a_indices = [parent_a[i][0] for i in range(1, len(parent_a))]
    parent_b_indices = [parent_b[i][0] for i in range(1, len(parent_b))]
    split_point = random.randint(0, min(len(parent_a_indices), len(parent_b_indices)) - 1)
    parent_a_indices[:split_point], parent_b_indices[split_point:] = parent_b_indices[:split_point], parent_a_indices[split_point:]
    for parent in (parent_a_indices, parent_b_indices):
        time_pointer = 0
        for i in range(len(parent)):
            if time_pointer >= days_number:
                parent_a_indices = parent[:i]
                break
            time_pointer += libraries[parent[i]][1]
        time_pointer = 0
    child_a = get_complete_solution(parent_a_indices, libraries, days_number, books_scores)
    child_b = get_complete_solution(parent_a_indices, libraries, days_number, books_scores)
    return (child_a, child_b)
    

In [7]:
def get_offspring(population, libraries, days_number, books_scores):
    offspring = []
    for i in range(len(population), 2):
        offspring.append(get_children(population[i], population[i + 1], libraries, days_number, books_scores))
    return tuple(offspring)

# Main

<font color="blue">__Specify how long program is able to run here (in seconds)__</blue>

In [26]:
max_duration = 200
epochs_duration = []
epochs_duration.append(time())
epochs_duration.append(time())

<font color="blue">__Specify population size here__</font>

In [27]:
#population_size = 40

### Read data

<font color=blue>__Specify path to input file here__</font>

In [28]:
instances = ["instances/a_example.txt", "instances/b_read_on.txt", "instances/c_incunabula.txt", "instances/d_tough_choices.txt", "instances/e_so_many_books.txt", "instances/f_libraries_of_the_world.txt"]

total_solution = 0
for i in instances:
    file = open(i, "r")
    
    #load data from the source code
    books_number, libraries_number, days_number = [int(x) for x in file.readline().split()]
        
    if(libraries_number <= 1000):
        population_size = 100
    elif(libraries_number <= 10000):
        population_size = 40
    else:
        population_size = 15

    books_scores = tuple([int(x) for x in file.readline().split()])
    libraries = [None] * libraries_number
    for j in range(libraries_number):
        libraries[j] = (tuple([int(x) for x in file.readline().split()]), tuple([int(x) for x in file.readline().split()]))
    libraries = tuple(libraries) 
    
    #get initial population
    population = get_initial_population(population_size, libraries_number, days_number, libraries, books_scores)

    #main loops - epochs
    epoch = 0
    while epochs_duration[0] + max_duration >= time() + (epochs_duration[-1] - epochs_duration[-2]) * 2:
        mutations = do_mutations(population, libraries, days_number, books_scores)
        offspring = get_offspring(population, days_number, libraries, books_scores)
        population.extend(offspring)
        population.extend(mutations)
        population.sort(key = lambda x: evaluate_solution(x, books_scores), reverse = True)
        population = population[:population_size]
        epochs_duration.append(time())
        #print("epoch: " + str(epoch) + "; timestamp: " + str(round(time() - epochs_duration[0], 2)) +  "; epoch's duration: ", str(round(epochs_duration[-1] - epochs_duration[-2], 2)) + "; best solution: " + str(evaluate_solution(population[0], books_scores)))
        epoch += 1
    
    sol = evaluate_solution(population[0], books_scores)
    print("For: ", i, "solution is: ", sol)
    
    total_solution += sol

For:  instances/a_example.txt solution is:  21
For:  instances/b_read_on.txt solution is:  4382200
For:  instances/c_incunabula.txt solution is:  868216
For:  instances/d_tough_choices.txt solution is:  4346550
For:  instances/e_so_many_books.txt solution is:  260459
For:  instances/f_libraries_of_the_world.txt solution is:  413114


In [29]:
print(total_solution)

10270560
